### edit distance

In [280]:
import numpy as np

def edit_distance(source, target, verbose=0, substitution_cost=1):
    source = "#" + source
    target = "#" + target
    graph = np.zeros((len(source), len(target)))
    graph[:, 0] = range(graph.shape[0])
    graph[0, :] = range(graph.shape[1])
    if verbose==1:
        print(graph)

    for i in range(1, len(source)):
        for j in range(1, len(target)):
            if source[i] != target[j]:
                if substitution_cost==1:
                    possible_values = np.asarray([graph[i-1, j], graph[i-1, j-1], graph[i, j-1]])
                elif substitution_cost==2:
                    possible_values = np.asarray([graph[i-1, j], graph[i, j-1]])
                graph[i, j] = 1 + np.min(possible_values)
            else:
                graph[i, j] = graph[i-1, j-1]

            if verbose==1:
                print(graph)
    return graph, int(graph[graph.shape[0]-1, graph.shape[1]-1])

In [281]:
graph, min_edit_distance = edit_distance("intention", "execution", verbose=0, substitution_cost=1)
graph

array([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
       [1., 1., 2., 3., 4., 5., 6., 6., 7., 8.],
       [2., 2., 2., 3., 4., 5., 6., 7., 7., 7.],
       [3., 3., 3., 3., 4., 5., 5., 6., 7., 8.],
       [4., 3., 4., 3., 4., 5., 6., 6., 7., 8.],
       [5., 4., 4., 4., 4., 5., 6., 7., 7., 7.],
       [6., 5., 5., 5., 5., 5., 5., 6., 7., 8.],
       [7., 6., 6., 6., 6., 6., 6., 5., 6., 7.],
       [8., 7., 7., 7., 7., 7., 7., 6., 5., 6.],
       [9., 8., 8., 8., 8., 8., 8., 7., 6., 5.]])

In [282]:
edit_distance("glowing", "growling", verbose=0, substitution_cost=1)

(array([[0., 1., 2., 3., 4., 5., 6., 7., 8.],
        [1., 0., 1., 2., 3., 4., 5., 6., 7.],
        [2., 1., 1., 2., 3., 3., 4., 5., 6.],
        [3., 2., 2., 1., 2., 3., 4., 5., 6.],
        [4., 3., 3., 2., 1., 2., 3., 4., 5.],
        [5., 4., 4., 3., 2., 2., 2., 3., 4.],
        [6., 5., 5., 4., 3., 3., 3., 2., 3.],
        [7., 6., 6., 5., 4., 4., 4., 3., 2.]]),
 2)

In [283]:
edit_distance("glad", "gal")

(array([[0., 1., 2., 3.],
        [1., 0., 1., 2.],
        [2., 1., 1., 1.],
        [3., 2., 1., 2.],
        [4., 3., 2., 2.]]),
 2)

### CKY parser

In [284]:
def cky_parser(string, grammar):
    table = np.zeros((len(string), len(string))).astype('str')
    table[:] = ""    
# Filling in the table 
    for j in range(0, len(table)): 

        # Iterate over the rules 
        for lhs, rule in grammar.items(): 
            for rhs in rule: 
                
                # If a terminal is found 
                if len(rhs) == 1 and rhs[0] == string[j]: 
                    table[j][j] += lhs
        # print(table)

        for i in range(j, -1, -1):    
            
            # Iterate over the range i to j + 1    
            for k in range(i, j + 1):      

                # Iterate over the rules 
                for lhs, rule in grammar.items(): 
                    for rhs in rule: 
                        # If a terminal is found 
                        if (k+1) < len(string):
                            if len(rhs) == 2 and rhs[0] in table[i][k] and rhs[1] in table[k+1][j]:
                                if lhs not in table[i][j]: 
                                    table[i][j] += lhs
                        else:
                            pass

    table = np.where(table=="", ".", table)

    for row in range(len(table)):
        for col in range(table.shape[1]):
            print(table[row][col], end="\t\t")
        print()

    if "S" in table[0][len(string)-1]:
        print("The string is VALID under the given grammar!")
        return table, True
    else:
        print("The string is INVALID under the given grammar!")
        return table, False

In [285]:
# print(cky_parser("abba", grammar))
grammar2 = {"S": ["AB", "AA", "CA"], "A": ["AB", "CB", "d", "q"], "B": ["BA", "CC", "c", "k"],
            "C": ["AA", "BB", "CC", "a", "p"]}
cky_parser("kkbapc", grammar2)
print()

B		C		.		.		.		.		
.		B		.		.		.		.		
.		.		.		.		.		.		
.		.		.		C		BC		SAC		
.		.		.		.		C		A		
.		.		.		.		.		B		
The string is INVALID under the given grammar!



In [286]:
# QUIZ 6 --- LAST EXERCISE
grammar = {"S": ["AB", "BC", "CC"], "A": ["CB", "AA", "l", "i"], 
           "B": ["AB", "l", "f", "p"], "C": ["BB", "BA", "p"]}
cky_parser("flip", grammar)
print()

B		C		SC		SCA		
.		AB		AC		SBCA		
.		.		A		SB		
.		.		.		BC		
The string is VALID under the given grammar!



In [287]:
grammar = ({"S": ["AB", "BC"], "A": ["BA", "a"], "B": ["CC", "b"], 
            "C": ["AB", "a"]})
cky_parser("abba", grammar),
print()

AC		SC		.		.		
.		B		.		A		
.		.		B		SA		
.		.		.		AC		
The string is INVALID under the given grammar!



In [288]:
grammar = {"S": ["AB", "BC", "CC"], "A": ["BA", "CB", "w", "x"],
           "B": ["CC", "x", "y", "z"], "C": ["AB", "BA", "AA", "y"]}
cky_parser("yyxzx", grammar)
print()

BC		SAB		SABC		SABC		SABC		
.		BC		AC		SBAC		SABC		
.		.		AB		SC		SAC		
.		.		.		B		AC		
.		.		.		.		AB		
The string is VALID under the given grammar!



In [289]:
grammar = {"S": ["AC", "BA", "A"], "A": ["Aa", "CC", "x", "y"], "B": ["BA", "CA", "x", "z"], "C": ["AA", "AB", "BB", "y"], "x": ["A"]}
cky_parser("xyaz", grammar)
print("-------------------------------------------------")

# mistakes:
# x --> A not in CNF
# A --> Aa | ... not in CNF
# output on Canvas cell 0,1 is C,S, should be C,S,B
# cells after row 2 should be empty, except for right corner

# CORRECT GRAMMAR
grammar = {"S": ["AC", "BA", "A"], "A": ["CC", "x", "y"], "B": ["BA", "CA", "x", "z"], "C": ["AA", "AB", "BB", "y"]}
cky_parser("xyaz", grammar)
print()

AB		SBC		.		.		
.		AC		.		.		
.		.		.		.		
.		.		.		B		
The string is INVALID under the given grammar!
-------------------------------------------------
AB		SBC		.		.		
.		AC		.		.		
.		.		.		.		
.		.		.		B		
The string is INVALID under the given grammar!



In [290]:
grammar = {"S": ["AB", "AA", "CA"], "A": ["AB", "CB", "d", "q"], "B": ["BA", "CC", "c", "k"], "C": ["AA", "BB", "CC", "a", "p"]}
cky_parser("kkbapc", grammar)
print()

B		C		.		.		.		.		
.		B		.		.		.		.		
.		.		.		.		.		.		
.		.		.		C		BC		SAC		
.		.		.		.		C		A		
.		.		.		.		.		B		
The string is INVALID under the given grammar!



### Viterbi algorithm

In [291]:
# TRANSITION AND EMISSION MATRIX
states = ('BOS', 'H', 'I', 'J', 'K', 'L', 'EOS')
observations = ('w', 'x', 'y', 'z')

t = {"BOS": [0, 0.8, 0.1, 0.1, 0, 0],
     "H": [0, 0.2, 0.4, 0.2, 0, 0.3],
     "I": [0.7, 0, 0, 0, 0.1, 0.2],
     "J": [0.1, 0.2, 0.2, 0.2, 0.2, 0.1],
     "K": [0.2, 0, 0.6, 0, 0.2, 0],
     "L": [0, 0.1, 0, 0.8, 0, 0.1]}

def create_matrix(t, els):
    dic = {}
    for i, el in zip(t, els):
        dic[el] = i
    return dic

transformation_matrix = {}
for state in states[:-1]:
    transformation_matrix[state] = create_matrix(t[state], states[1:])

e = {"H": [0.2, 0, 0, 0.8], 
     "I": [0, 1, 0, 0],
     "J": [0.5, 0.1, 0.4, 0],
     "K": [0, 0, 0.9, 0.1],
     "L": [0, 0, 0.5, 0.5]}

emission_matrix = {}
for state in states[1:-1]:
    emission_matrix[state] = create_matrix(e[state], observations)

def viterbi_algorithm(t, e,  string, bos='BOS', eos='EOS'):

    """
    Implementation of the Viterbi algorithm for PoS tagging

    Parameters
    ----------
    t: transition matrix. Contains the probability of a tag given another tag
    e: emission matrix. Contains the probability of a word given a tag
    string: input string for which to compute the likeliest sequence of tags. Despite its name, this can also be a list of words representing a \
        sentence
    bos: string representation of the BoS symbol. Default is 'BOS'
    eos: string representation of the EoS symbol. Default is 'EOS'

    Returns
    -------
    The final matrix containing the final posterior probabilities, and a sequence containing the maximum a posteriori sequence of tags

    Notes
    -----
    In the transition matrix, the first state (i.e., the first key) has to be the BoS state, otherwise the implementation will return a wrong result. \
    The same goes for the EoS symbol, this has to be the last key in the transition matrix. The values in the emission matrix have to correspond to \
    the order in which the observations are given in the emission matrix. For example, if the probabilities for state A are 0, 0.2, and 0.8 and the \
    order of observations in the emission matrix is a, b, c, then 0 has to correspond to a|A, 0.2 to b|A, and 0.8 to c|A. Hence, carefully check the \
    way in which the data is formatted before using the algorithm.
    """

    # column names -- for the columns of the final output matrix
    col_names = []

    # states (excluding BoS)
    states = [key for key in list(t.keys()) if key != bos]

    # observations
    observations = [key for key in list(e.keys())]

    col_names.append(bos) # add BoS symbol to column names
    col_names.extend(string) # add individual input elements to column names
    col_names.append(eos) # add EoS symbol to column names

    # final output matrix -- initialize with zeros
    matrix = np.zeros((len(states), len(col_names)))

    # for each observation, store observation (key) and add conditional probability of observation given each tag in value (a list)
    observation_dict = dict()

    for key in list(e.keys()):
        for k in list(e[key].keys()):
            observation_dict[k] = []
    
    for key in list(e.keys()):
        for k in list(e[key].keys()):
            observation_dict[k].append(e[key][k])
    
    # all conditional probabilities for the BoS state given each state, except EoS (the last state)
    bos_values = np.asarray([value for value in list(t[bos].values())[:-1]])

    # all conditional probabilities for the EoS state given each state
    eos_values = np.asarray([t[state][eos] for state in states])

    # add the BoS conditional probability values to the observation-probabilities dictionary, as BoS is also an observation
    observation_dict[bos] = bos_values

    # make all conditional probability values for the observation an array for easier and faster computation
    observation_dict = {key: np.asarray(value) for key, value in observation_dict.items()}

    # first column in final output matrix should be the conditional probability values for the BoS state
    matrix[:, 0] = bos_values

    # select only parts of the transformation matrix necessary for computation for all non-BoS and non-EoS states (so exclude EoS column)
    useful_matrix = []

    for state in states:
        useful_matrix.append([value for value in list(t[state].values())][:-1])

    useful_matrix = np.asarray(useful_matrix)

    # second column should be first column (i.e., BoS probabilities) multiplied by the column observation probabilities
    matrix[:, 1] = matrix[:, 0] * observation_dict[string[0]]

    # do the calculations, with the previous column being the trellis
    for i in range(1, len(string)):

        values = observation_dict[string[i]] # probabilities for symbol we are currently looping over
        results = np.zeros((useful_matrix.shape)) # store all multiplication results in an array, intitialize with zeros
        trellis = matrix[:, i] # select the trellis -- the previous posterior probabilities

        # loop over all rows
        for j in range(len(results)):
            results[j, :] = trellis[j] * useful_matrix[j, :] # row in resulting matrix should each individual trellis value multiplied by the 
                                                             # transformation matrix row

        max_ = np.max(results) # select maximum value of the results
        matrix[:, i+1] = max_ * values # multiply the observation probabilities by this maximum value and store in next column
    
    # the final values for the EoS observation should be the product of the values of the last non-EoS observation and the values of the EoS observation
    matrix[:, -1] = matrix[:, -2] * eos_values

    # print matrix values
    for i in range(len(matrix)):
        for el in matrix[i, :]:
            print(round(el, 10), end="\t\t")
        print()
    print("\n")
    
    # the final sequence of which state most likely belongs to each observation
    sequence = {}
    for i, name in enumerate(col_names):
        sequence[name] = states[np.argmax(matrix[:, i])]

    # print the final sequence
    print("Solution:", end=" ")

    for key, value in sequence.items():
        if key != eos:
            print(f"{key} ---> {value}", end=", ")
        else:
            print(f"{key} ---> {value}")

    # return the output matrix and the final sequence
    return matrix, sequence

results = viterbi_algorithm(transformation_matrix, emission_matrix, string="xwxy")

print("Next ... ")

results2 = viterbi_algorithm(transformation_matrix, emission_matrix, string="wzyx")



0.0		0.0		0.112		0.0		0.0		0.0		
0.8		0.8		0.0		0.056		0.0		0.0		
0.1		0.01		0.28		0.0056		0.01568		0.001568		
0.1		0.0		0.0		0.0		0.03528		0.0		
0.0		0.0		0.0		0.0		0.0196		0.00196		


Solution: BOS ---> I, x ---> I, w ---> J, y ---> K, EOS ---> L
Next ... 
0.0		0.0		0.008		0.0		0.0		0.0		
0.8		0.0		0.0		0.0		0.00216		0.000432		
0.1		0.05		0.0		0.0016		0.000216		2.16e-05		
0.1		0.0		0.001		0.0036		0.0		0.0		
0.0		0.0		0.005		0.002		0.0		0.0		


Solution: BOS ---> I, w ---> J, z ---> H, y ---> K, x ---> I, EOS ---> I


In [292]:
# TRANSFORMATION AND EMISSION MATRIX
states = ('BOS', 'Det', 'Adj', 'Noun', 'Verb', 'EOS')
observations = ('dog', 'the', 'chases', 'fat', 'cat')

t = {"BOS": [0.5, 0.2, 0.3, 0, 0],
     "Det": [0, 0.2, 0.8, 0, 0],
     "Adj": [0, 0.3, 0.6, 0, 0.1],
     "Noun": [0, 0, 0, 0.5, 0.5],
     "Verb": [0.5, 0.2, 0.3, 0, 0]}

def create_matrix(t, els):
    dic = {}
    for i, el in zip(t, els):
        dic[el] = i
    return dic

transformation_matrix = {}
for state in states[:-1]:
    transformation_matrix[state] = create_matrix(t[state], states[1:])

e = {"Det": [0, 1, 0, 0, 0],
     "Adj": [0, 0, 0, 0, 1],
     "Noun":[0.5, 0, 0, 0.4, 0.1],
     "Verb": [0.1, 0, 0.8, 0.1, 0]}

emission_matrix = {}
for state in states[1:-1]:
    emission_matrix[state] = create_matrix(e[state], observations)




results = viterbi_algorithm(transformation_matrix, emission_matrix, string=["the", "dog", "chases", "the", "fat", "cat"])

print(transformation_matrix)
print(emission_matrix)

0.5		0.5		0.0		0.0		0.04		0.0		0.0		0.0		
0.2		0.0		0.0		0.0		0.0		0.0		0.0064		0.00064		
0.3		0.0		0.2		0.0		0.0		0.0128		0.00064		0.00032		
0.0		0.0		0.04		0.08		0.0		0.0032		0.0		0.0		


Solution: BOS ---> Det, the ---> Det, dog ---> Noun, chases ---> Verb, fat ---> Noun, cat ---> Adj, EOS ---> Adj
{'BOS': {'Det': 0.5, 'Adj': 0.2, 'Noun': 0.3, 'Verb': 0, 'EOS': 0}, 'Det': {'Det': 0, 'Adj': 0.2, 'Noun': 0.8, 'Verb': 0, 'EOS': 0}, 'Adj': {'Det': 0, 'Adj': 0.3, 'Noun': 0.6, 'Verb': 0, 'EOS': 0.1}, 'Noun': {'Det': 0, 'Adj': 0, 'Noun': 0, 'Verb': 0.5, 'EOS': 0.5}, 'Verb': {'Det': 0.5, 'Adj': 0.2, 'Noun': 0.3, 'Verb': 0, 'EOS': 0}}
{'Det': {'dog': 0, 'the': 1, 'chases': 0, 'fat': 0, 'cat': 0}, 'Adj': {'dog': 0, 'the': 0, 'chases': 0, 'fat': 0, 'cat': 1}, 'Noun': {'dog': 0.5, 'the': 0, 'chases': 0, 'fat': 0.4, 'cat': 0.1}, 'Verb': {'dog': 0.1, 'the': 0, 'chases': 0.8, 'fat': 0.1, 'cat': 0}}


In [293]:
states = ('BOS', 'A', 'B', 'C', 'EOS')
observations = ('v', 'w', 'x', 'y', 'z')

t = {"BOS": [0.7, 0.3, 0, 0],
     "A": [0, 0.2, 0.8, 0],
     "B": [0.2, 0.3, 0.4, 0.1],
     "C": [0.1, 0.2, 0.2, 0.5]}

def create_matrix(t, els):
    dic = {}
    for i, el in zip(t, els):
        dic[el] = i
    return dic

transformation_matrix = {}
for state in states[:-1]:
    transformation_matrix[state] = create_matrix(t[state], states[1:])

e = {"A": [0, 0.8, 0, 0.2, 0],
     "B": [0.4, 0.1, 0.5, 0, 0],
     "C":[0, 0, 0.3, 0.1, 0.6]}

emission_matrix = {}
for state in states[1:-1]:
    emission_matrix[state] = create_matrix(e[state], observations)




results = viterbi_algorithm(transformation_matrix, emission_matrix, string="wvy")

0.7		0.56		0.0		0.014336		0.0		
0.3		0.03		0.1792		0.0		0.0		
0.0		0.0		0.0		0.007168		0.003584		


Solution: BOS ---> A, w ---> A, v ---> B, y ---> A, EOS ---> C
